In [1]:
# Quelques imports utiles
import random
import utils

## Lecture du corpus

In [2]:
import random
from utils import preprocess

with open("data/Les_Misérables.txt", "r", encoding="utf8") as f:
    corpus = f.read()

max_sentences = 1000
corpus = preprocess(corpus)[:max_sentences]

for doc in random.sample(corpus, 5):
    print(*doc)

il échappait toujours quelquefois il résistait de vive force
pendant tout le temps qu il occupa le siège de digne m
en rentrant il dînait
myriel vous regardez un bonhomme et moi je regarde un grand homme
on mit la gendarmerie à ses trousses mais en vain


In [3]:
# Quelques 2-grammes
bigrams = [line.split() for line in 
"""<s> <s>
cosette était
elle était
je suis
sombre imbécile
c est""".split("\n")]

trigrams = [line.split() for line in 
"""<s> <s> il
cosette était sortie
elle était toujours
je suis fort
sombre imbécile couac
c est un""".split("\n")]

Quelques fonctions de test et de démonstration :

In [4]:
def demo(model):
    print("------------\nPROBABILITÉS\n------------")
    for *context, word in trigrams:
        print("P", word, "|", *context, "=", model.proba(word, context))

    print("\n\n-----------\nPRÉDICTIONS\n-----------")

    for context in bigrams:
        print(*context, "->", model.predict_next(context))

    print("\n\n-------------------\nN-GRAMMES FRÉQUENTS\n-------------------")
    for n in (2, 3):
        print("\nn =", n)
        for ngram, count in model.counts[n].most_common(5):
            print(*ngram, count)
    
    print("\n\n----------\nGÉNÉRATION\n----------")
    for _ in range(5):
        print(*model.generate(max_words=100))

## Maximum de vraisemblance

L'estimateur à maximum de vraisemblance EMV (en anglais : MLE, pour *Maximum Likelihood Estimator*).

Soit $D = w_1 \ldots w_N$ un corpus composés de $N$ mots. Modèle de langue $n$-gramme : soit $n \geq 1$, et $w_i$ un mot. Son contexte est composé des $n-1$ mots qui le précèdent : $c_i = w_{i-n+1}, \ldots, w_{i-1}$.

$$
P_{MLE}(w_i | c_i) = \frac{|c_i, w_i|}{|c_i|}
$$

In [5]:
from models.mle import MLE

mle = MLE(corpus, 3)

demo(mle)

------------
PROBABILITÉS
------------
P il | <s> <s> = 0.133
P sortie | cosette était = 0.0
P toujours | elle était = 0.0
P fort | je suis = 0.0
P couac | sombre imbécile = 0.0
P un | c est = 0.09523809523809523


-----------
PRÉDICTIONS
-----------
<s> <s> -> s
cosette était -> science
elle était -> atteinte
je suis -> tranquille
sombre imbécile -> delà
c est -> ce


-------------------
N-GRAMMES FRÉQUENTS
-------------------

n = 2
<s> <s> 1000
<s> il 133
l évêque 94
de la 70
qu il 58

n = 3
<s> <s> il 133
<s> <s> l 56
<s> <s> le 51
<s> l évêque 35
<s> <s> c 34


----------
GÉNÉRATION
----------
<s> <s> géborand n avait de l évêque visita l hôpital je vais vous faire un petit carré de papier jauni par le juge et par le roi l un à dîner madame magloire a découvert sous au moins dix papiers collés dessus des peintures sans être bonnes qui peuvent se supporter </s>
<s> <s> il advint qu un évêque se borna à dire je renoncerais difficilement à manger en cérémonie le 29 juillet 1714 y éta

In [6]:
utils.inspect(mle.proba)

```python
    def proba(self, word: str, context: List[str]):
        p_context = self.counts[context]
        if p_context:
            return self.counts[(*context, word)] / p_context
        return 0.0
```

## Laplace et Lidstone (lissage additif)

In [7]:
from models.lidstone import Lidstone

lid = Lidstone(corpus, 3, gamma=0.1)

demo(lid)

------------
PROBABILITÉS
------------
P il | <s> <s> = 0.10055148447533428
P sortie | cosette était = 0.0003089280197713932
P toujours | elle était = 0.0003070310101320233
P fort | je suis = 0.0002996703626011387
P couac | sombre imbécile = 0.0003089280197713932
P un | c est = 0.011211375444353293


-----------
PRÉDICTIONS
-----------
<s> <s> -> je
cosette était -> brouettes
elle était -> rachats
je suis -> toucha
sombre imbécile -> jour
c est -> tout


-------------------
N-GRAMMES FRÉQUENTS
-------------------

n = 2
<s> <s> 1000
<s> il 133
l évêque 94
de la 70
qu il 58

n = 3
<s> <s> il 133
<s> <s> l 56
<s> <s> le 51
<s> l évêque 35
<s> <s> c 34


----------
GÉNÉRATION
----------
<s> <s> toutes idées vain détrempe apercevrez bonnes avenir horreur étroit actions renouvelle vint 29 souvent portes perplexe prouvent casse nouveau passivement part depuis queyras longues double totale réunir voiture prières rentre dangers appelons décrets lugubres faut réflexion président reculer tournan

## Good-Turing


$$
r^* = (r + 1) \times \frac{n_{r+1}}{n_r}
$$

In [8]:
from models.good_turing import GoodTuring

gt = GoodTuring(corpus, 3)

demo(gt)

------------
PROBABILITÉS
------------
P il | <s> <s> = 0.133
P sortie | cosette était = 0.00044851824412775693
P toujours | elle était = 4.2339620906523123e-07
P fort | je suis = 8.356830170147243e-08
P couac | sombre imbécile = 0.00044851824412775693
P un | c est = 0.055114638447971785


-----------
PRÉDICTIONS
-----------
<s> <s> -> alors
cosette était -> certains
elle était -> à
je suis -> néant
sombre imbécile -> rivale
c est -> un


-------------------
N-GRAMMES FRÉQUENTS
-------------------

n = 2
<s> <s> 1000
<s> il 133
l évêque 94
de la 70
qu il 58

n = 3
<s> <s> il 133
<s> <s> l 56
<s> <s> le 51
<s> l évêque 35
<s> <s> c 34


----------
GÉNÉRATION
----------
<s> <s> il lui faut du luxe comme aux anciens évêques </s>
<s> <s> l évêque </s>
<s> <s> ce sont mes bons amis les bergers </s>
<s> <s> un jour il se nommait g </s>
<s> <s> c est la règle </s>
